In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os
import polars as pl

import yaml
with open('config.yaml','r') as file:
    config=yaml.safe_load(file)
dataset_type="1d_raw_std"
target_type="gas"
x_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_train_{dataset_type}.npy")
t_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_train_{dataset_type}.npy")
# x_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/x_test_{dataset_type}.npy")
# t_test=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/dataset/npy/tg_test_{dataset_type}.npy")

bdir="/mnt/sdb/yyamaguchi/simulationB4/dataset"
odir=os.path.join(bdir,f"outputs_{dataset_type}_{target_type}")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B     = config['hyperparameter']['batch_size']
W     = config['hyperparameter']['input_length']
H     = config['hyperparameter']['input_height']
lr    = config['hyperparameter']['learning_rate']
N     = config['hyperparameter']['num_epochs']
l2    = config['hyperparameter']['l2_lambda']
delta = config['hyperparameter']['huber_delta']
C     = config['hyperparameter']['input_channel']

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

class SimpleCNN2d(nn.Module):
    def __init__(self, input_width, input_height, input_channel, mid_channel=16):
        super(SimpleCNN2d, self).__init__()
        self.input_channel = input_channel
        self.input_width = input_width
        self.input_height = input_height
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv2d(1, mid_channel, kernel_size=16),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(mid_channel, mid_channel, kernel_size=4),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

# class SimpleCNN(nn.Module):
#     def __init__(self, input_length, input_channel, mid_channel=16):
#         super(SimpleCNN, self).__init__()
#         self.input_channel = input_channel
#         self.input_length = input_length
#         self.mid_channel = mid_channel
#         self.conv1 = nn.Conv1d(1, mid_channel, kernel_size=201)
#         self.bn1 = nn.BatchNorm1d(mid_channel)
#         self.maxpool1 = nn.MaxPool1d(4,stride=1)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv1d(mid_channel, mid_channel, kernel_size=51)
#         self.bn2 = nn.BatchNorm1d(mid_channel)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool1d(4,stride=1)
#         self.pool = nn.AdaptiveAvgPool1d(1)
#         self.fc = nn.Linear(mid_channel*input_channel, 1)
#         self.dropout = nn.Dropout(0.4)
#         # self.ln1 = nn.LayerNorm([16, input_length])
#         # self.ln2 = nn.LayerNorm([4, input_length+2])
#     def forward(self, xall):
#         bsize = xall.shape[0]
#         output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
#         for i in range(self.input_channel):
#             x = xall[:,i,:]
#             x = x.unsqueeze(1)
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu1(x)
#             x = self.maxpool1(x)
#             # x = self.ln1(x)
#             x = self.conv2(x)
#             x = self.bn2(x)
#             x = self.relu2(x)
#             x = self.maxpool2(x)
#             # x = self.ln2(x)
#             x = self.pool(x)
#             x = self.dropout(x)
#             x = x.view(x.size(0), -1)
#             output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
#         output = self.fc(output)
#         return output.squeeze(1)

print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize
rng=np.random.default_rng()
# xt=torch.from_numpy(x_test).float()
# tt=torch.from_numpy(t_test).float()
# testset=TensorDataset(xt,tt)
# testloader=DataLoader(testset,batch_size=1,shuffle=False)
# testsize=len(testset)

for k in range(10):
    testresult=np.zeros((2,testsize))
    trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=1,shuffle=False)
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    model=SimpleCNN(W,C).to(device)
    # params = 0
    # for p in model.parameters():
    #     if p.requires_grad:
    #         params += p.numel()
            
    # print(params)

    criterion=nn.HuberLoss(delta=delta)
    # criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            # start_idx=rng.integers(250)
            start_idx=0
            bx=bx[:,:,start_idx:start_idx+W]
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            # l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l2*l2n*0.5
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx[:,:,:W]
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx[:,:,:W]
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    testresult[0,:]=targ
    testresult[1,:]=pred
    np.save(os.path.join(wdir,f'result{k}.npy'),testresult)

    corr=np.corrcoef(targ,pred)
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    if target_type == "gas":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.4)
        plt.ylim(0, 0.4)
        plt.xticks([0,0.1,0.2,0.3,0.4])
        plt.yticks([0,0.1,0.2,0.3,0.4])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()
    if target_type == "solid":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.06)
        plt.ylim(0, 0.06)
        plt.xticks([0,0.02,0.04,0.06])
        plt.yticks([0,0.02,0.04,0.06])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()

reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")
df = pl.DataFrame({
    "RMSE":reg_list,
    "R2":coefg_list
})
mean = df.mean()
std = df.std()

df = pl.concat([
    df,
    mean,
    std
])
df.write_csv(os.path.join(odir,"result.csv"))

(605, 2, 3000)


/tmp/ipykernel_3580120/433510039.py:258: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


coefficient void: 0.22251630318990268
re void: 0.01415292249122316
coefficient void: 0.2993000486567635
re void: 0.01577356319409036
coefficient void: 0.10676467926155782
re void: 0.01651369375802633
coefficient void: 0.2370725910119064
re void: 0.013988928729843293
coefficient void: 0.3541958521843564
re void: 0.014851504652820948
coefficient void: 0.23385982125398472
re void: 0.019423056594722477
coefficient void: 0.4046976938639121
re void: 0.01539729604465758
coefficient void: 0.08076455743509421
re void: 0.01754540297970157
coefficient void: 0.0388416983258192
re void: 0.016684603751680144
coefficient void: 0.38354176545761914
re void: 0.013053321820990364
reg_list:[0.01415292 0.01577356 0.01651369 0.01398893 0.0148515  0.01942306
 0.0153973  0.0175454  0.0166846  0.01305332]
coefg_list:[0.2225163  0.29930005 0.10676468 0.23707259 0.35419585 0.23385982
 0.40469769 0.08076456 0.0388417  0.38354177]
reg:0.015738429401775625
coefg:0.2361555010640916


In [18]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os
import polars as pl

dataset_type="two_raw_std"
target_type="solid"
x_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_{dataset_type}.npy")
t_train=np.load(f"/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/ts_train_{dataset_type}.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,f"outputs_{dataset_type}_{target_type}")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=32
W=2500
H=49
lr=0.01
N=50
l1=1e-7
l2=1e-6
delta=0.01
C=2

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

class SimpleCNN2d(nn.Module):
    def __init__(self, input_width, input_height, input_channel, mid_channel=16):
        super(SimpleCNN2d, self).__init__()
        self.input_channel = input_channel
        self.input_width = input_width
        self.input_height = input_height
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, mid_channel):
        return nn.Sequential(
            nn.Conv2d(1, mid_channel, kernel_size=16),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(mid_channel, mid_channel, kernel_size=4),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

# class SimpleCNN(nn.Module):
#     def __init__(self, input_length, input_channel, mid_channel=16):
#         super(SimpleCNN, self).__init__()
#         self.input_channel = input_channel
#         self.input_length = input_length
#         self.mid_channel = mid_channel
#         self.conv1 = nn.Conv1d(1, mid_channel, kernel_size=201)
#         self.bn1 = nn.BatchNorm1d(mid_channel)
#         self.maxpool1 = nn.MaxPool1d(4,stride=1)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv1d(mid_channel, mid_channel, kernel_size=51)
#         self.bn2 = nn.BatchNorm1d(mid_channel)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool1d(4,stride=1)
#         self.pool = nn.AdaptiveAvgPool1d(1)
#         self.fc = nn.Linear(mid_channel*input_channel, 1)
#         self.dropout = nn.Dropout(0.4)
#         # self.ln1 = nn.LayerNorm([16, input_length])
#         # self.ln2 = nn.LayerNorm([4, input_length+2])
#     def forward(self, xall):
#         bsize = xall.shape[0]
#         output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
#         for i in range(self.input_channel):
#             x = xall[:,i,:]
#             x = x.unsqueeze(1)
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu1(x)
#             x = self.maxpool1(x)
#             # x = self.ln1(x)
#             x = self.conv2(x)
#             x = self.bn2(x)
#             x = self.relu2(x)
#             x = self.maxpool2(x)
#             # x = self.ln2(x)
#             x = self.pool(x)
#             x = self.dropout(x)
#             x = x.view(x.size(0), -1)
#             output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
#         output = self.fc(output)
#         return output.squeeze(1)

print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize

for k in range(10):
    testresult=np.zeros((2,testsize))
    trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=1,shuffle=False)
    model=SimpleCNN(W,C).to(device)
    # params = 0
    # for p in model.parameters():
    #     if p.requires_grad:
    #         params += p.numel()
            
    # print(params)

    criterion=nn.HuberLoss(delta=delta)
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            # l1n=sum(p.abs().sum() for p in model.parameters())
            # l2n=sum(p.pow(2).sum() for p in model.parameters())
            # loss=loss+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    testresult[0,:]=targ
    testresult[1,:]=pred
    np.save(os.path.join(wdir,f'result{k}.npy'),testresult)

    corr=np.corrcoef(targ,pred)
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    if target_type == "gas":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.4)
        plt.ylim(0, 0.4)
        plt.xticks([0,0.1,0.2,0.3,0.4])
        plt.yticks([0,0.1,0.2,0.3,0.4])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()
    if target_type == "solid":
        plt.figure(figsize=(8, 7))
        plt.scatter(targ, pred, alpha=0.6, marker='o', label='Phase fraction')
        plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
        plt.title('Predicted vs Actual Values')
        plt.xlabel('Actual Value')
        plt.ylabel('Predicted Value')
        plt.xlim(0, 0.06)
        plt.ylim(0, 0.06)
        plt.xticks([0,0.02,0.04,0.06])
        plt.yticks([0,0.02,0.04,0.06])
        plt.rcParams["font.size"] = 20
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
        plt.close()

reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")
df = pl.DataFrame({
    "RMSE":reg_list,
    "R2":coefg_list
})
mean = df.mean()
std = df.std()

df = pl.concat([
    df,
    mean,
    std
])
df.write_csv(os.path.join(odir,"result.csv"))

(605, 2, 2500)


/tmp/ipykernel_3438451/2932046310.py:241: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


coefficient void: 0.3469683908191165
re void: 0.018440722598173864


KeyboardInterrupt: 

In [30]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_two_env_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/tg_train_two_env_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_2env_std")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=32
W=2500
lr=0.001
N=50
l1=1e-7
l2=1e-6
C=2

class SimpleCNN(nn.Module):
    def __init__(self, input_length, input_channel, mid_channel=16):
        super(SimpleCNN, self).__init__()
        self.input_channel = input_channel
        self.input_length = input_length
        self.mid_channel = mid_channel
        self.models = nn.ModuleList([
            self._build_sub_network(input_length,mid_channel) for _ in range(input_channel)
        ])
        self.fc = nn.Linear(input_channel*mid_channel,1)
        # self.ln1 = nn.LayerNorm([mid_channel, int(input_length/5)])
        # self.ln2 = nn.LayerNorm([mid_channel, int(input_length/25)])
    def _build_sub_network(self, input_length, mid_channel):
        return nn.Sequential(
            nn.Conv1d(1, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(mid_channel, mid_channel, kernel_size=201),
            nn.BatchNorm1d(mid_channel),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.AdaptiveAvgPool1d(1),
            nn.Dropout(0.1)
        )

    def forward(self, xall):
        bsize = xall.shape[0]
        output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
        for i in range(self.input_channel):
            x = xall[:,i]
            x = x.unsqueeze(1)
            x = self.models[i](x)
            x = x.view(x.size(0), -1)
            output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
        output = self.fc(output)
        return output.squeeze(1)

# class SimpleCNN(nn.Module):
#     def __init__(self, input_length, input_channel, mid_channel=16):
#         super(SimpleCNN, self).__init__()
#         self.input_channel = input_channel
#         self.input_length = input_length
#         self.mid_channel = mid_channel
#         self.conv1 = nn.Conv1d(1, mid_channel, kernel_size=201)
#         self.bn1 = nn.BatchNorm1d(mid_channel)
#         self.maxpool1 = nn.MaxPool1d(4,stride=1)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv1d(mid_channel, mid_channel, kernel_size=51)
#         self.bn2 = nn.BatchNorm1d(mid_channel)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool1d(4,stride=1)
#         self.pool = nn.AdaptiveAvgPool1d(1)
#         self.fc = nn.Linear(mid_channel*input_channel, 1)
#         self.dropout = nn.Dropout(0.4)
#         # self.ln1 = nn.LayerNorm([16, input_length])
#         # self.ln2 = nn.LayerNorm([4, input_length+2])
#     def forward(self, xall):
#         bsize = xall.shape[0]
#         output = torch.zeros(bsize,self.input_channel*self.mid_channel).to(device)
#         for i in range(self.input_channel):
#             x = xall[:,i,:]
#             x = x.unsqueeze(1)
#             x = self.conv1(x)
#             x = self.bn1(x)
#             x = self.relu1(x)
#             x = self.maxpool1(x)
#             # x = self.ln1(x)
#             x = self.conv2(x)
#             x = self.bn2(x)
#             x = self.relu2(x)
#             x = self.maxpool2(x)
#             # x = self.ln2(x)
#             x = self.pool(x)
#             x = self.dropout(x)
#             x = x.view(x.size(0), -1)
#             output[:,self.mid_channel*i:self.mid_channel*(i+1)] = x
#         output = self.fc(output)
#         return output.squeeze(1)

class GradCAM1d:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def __call__(self, input_tensor):
        self.model.eval()
        input_tensor = input_tensor.to(next(self.model.parameters()).device)
        self.model.zero_grad()
        out = self.model(input_tensor)
        print(out)
        if isinstance(out, (tuple, list)):
            out = out[0]
        target = out.squeeze()
        print(target)
        if target.ndim > 0:
            target = target.sum()
        self.model.zero_grad()
        target.backward(retain_graph=True)
        gradients = self.gradients         # [B, C, L]
        print(gradients.shape)
        activations = self.activations     # [B, C, L]
        print(activations.shape)
        weights = gradients.mean(dim=2, keepdim=True)  # [B, C, 1]
        grad_cam_map = (weights * activations).sum(dim=1, keepdim=True)  # (B,1,L)
        grad_cam_map = torch.relu(grad_cam_map)
        grad_cam_map = torch.nn.functional.interpolate(
            grad_cam_map, size=input_tensor.shape[2], mode='linear', align_corners=False
        )
        grad_cam_map = grad_cam_map.squeeze().cpu().numpy()
        grad_cam_map = (grad_cam_map - grad_cam_map.min()) / (grad_cam_map.max() - grad_cam_map.min() + 1e-8)
        return grad_cam_map

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

print(x_train.shape)
# x_train=x_train.reshape(x_train.shape[0],-1)
# x_train=x_train[:,[1,3,4],:]
print(t_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
# x=x.unsqueeze(1)
print(x.shape)
amp=10
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
xgradcam=x[50,:,:]
xgradcam=xgradcam.unsqueeze(0)
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize

for k in range(10):
    trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
    testloader=DataLoader(testset,batch_size=1,shuffle=False)
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=1,shuffle=False)
    model=SimpleCNN(W,C).to(device)
    params = 0
    for p in model.parameters():
        if p.requires_grad:
            params += p.numel()
            
    print(params)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            # l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    print(pred.shape)
    print(targ.shape)

    # corr=np.corrcoef(targ[:,0],pred[:,0])
    # coef=corr[0,1]
    # print(f'coefficient solid: {coef}')
    # # re=np.mean(np.abs(targ[:,0]-pred[:,0])/(targ[:,0]+1e-7))
    # re=np.sqrt(np.sum(np.power(targ[:,0]-pred[:,0],2))/targ.shape[0])
    # print(f're solid: {re}')
    # res_list.append(re)
    # coefs_list.append(coef)
    # corr=np.corrcoef(targ[:,1],pred[:,1])
    # coef=corr[0,1]
    # print(f'coefficient void: {coef}')
    # # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    # re=np.sqrt(np.sum(np.power(targ[:,1]-pred[:,1],2))/targ.shape[0])
    # print(f're void: {re}')
    # reg_list.append(re)
    # coefg_list.append(coef)

    # plt.figure(figsize=(8, 8))
    # plt.scatter(targ[:,0], pred[:,0], alpha=0.6, marker='o', label='Solid fraction')
    # plt.plot([-1, 1], [-1, 1], 'r--', label='Ideal (y=x)')
    # plt.title('Predicted vs Actual Values')
    # plt.xlabel('Actual Value')
    # plt.ylabel('Predicted Value')
    # plt.xlim(-0.0, 0.05)
    # plt.ylim(-0.0, 0.05)
    # plt.rcParams["font.size"] = 20
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
    # plt.close()

    # plt.figure(figsize=(8, 8))
    # plt.scatter(targ[:,1], pred[:,1], alpha=0.6, marker='o', label='Void fraction')
    # plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    # plt.title('Predicted vs Actual Values')
    # plt.xlabel('Actual Value')
    # plt.ylabel('Predicted Value')
    # plt.xlim(0, 0.4)
    # plt.ylim(0, 0.4)
    # plt.rcParams["font.size"] = 20
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
    # plt.close()

    corr=np.corrcoef(targ,pred)
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ-pred,2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    plt.figure(figsize=(8, 8))
    plt.scatter(targ, pred, alpha=0.6, marker='o', label='Void fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual{k}.png'))
    plt.close()

    # target_layer = None
    # if hasattr(model, 'conv3'):
    #     target_layer = model.conv3
    # elif hasattr(model, 'layer3'):
    #     target_layer = model.layer3
    # else:
    #     for m in reversed(list(model.modules())):
    #         if isinstance(m, torch.nn.Conv1d):
    #             target_layer = m
    #             break
    # if target_layer is None:
    #     raise RuntimeError("Could not find a suitable layer for Grad-CAM.")

    # gradcam_output_dir = os.path.join(odir, "gradcam")
    # gradcam = GradCAM1d(model, target_layer)
    # grad_cam_map = gradcam(xgradcam)
    # gradcam.remove_hooks()

    # plt.figure(figsize=(10,6))
    # plt.rcParams['font.size'] = 20
    # amp=10
    # ta=np.arange(W)
    # xgradcam=xgradcam.detach().cpu()
    # xgradcamplt=xgradcam.numpy().squeeze(0)
    # plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    # plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    # # plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    # # plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    # # plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    # grad_cam_map *= 30
    # grad_cam_map -= amp
    # plt.legend(loc='upper right')
    # plt.plot(ta,grad_cam_map[:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, grad_cam_map[:],-amp,
    #         color='red',alpha=0.1)
    # plt.xlabel('Time Axis')
    # plt.ylabel('Pressure [-]')
    # plt.ylim(-amp,2*amp)
    # plt.yticks([-amp,0,amp,2*amp],[f"{-amp}","0","0",f"{amp}"])
    # # plt.ylim(-amp,5*amp)
    # # plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    # plt.xlim(0,W)
    # plt.grid(True)
    # # plt.xlim(35,40)
    # plt.tight_layout()
    # new_save_path = os.path.join(odir, f"gradcam{k}.png")
    # plt.savefig(new_save_path)
    # plt.close()

    # model.eval()
    # model.zero_grad()

    # # 1. まずデバイスに送る
    # xgradcam = xgradcam.to(device)

    # # 2. デバイス移動「後」に勾配追跡をONにする
    # xgradcam.requires_grad_()

    # # 3. 予測
    # output = model(xgradcam)

    # # 4. 逆伝播（クラス1に注目する場合）
    # loss = output[0]
    # loss.backward()

    # # 5. 勾配を取り出す（.gradプロパティに格納されている）
    # if xgradcam.grad is not None:
    #     grads = xgradcam.grad.data # ここで [1, 5, 2500] が取得できる
        
    #     # --- チャネル別重要度スコアの計算 ---
    #     # abs()をとって時間方向に平均
    #     salimap = grads.abs().squeeze(0)
        
    #     # 0~1に正規化（安全のために 1e-8 を足す）
    #     salimap = salimap / (salimap.max() + 1e-8)
        
    #     salimap = salimap.cpu().numpy()
    #     salimap = salimap*10
    #     print(salimap.shape)
    # else:
    #     print("勾配が取得できませんでした。requires_gradの位置を確認してください。")


    # plt.figure(figsize=(10,6))
    # plt.rcParams['font.size'] = 20
    # amp=10
    # xgradcam=xgradcam.detach().cpu()
    # xgradcamplt=xgradcam.numpy().squeeze(0)
    # plt.plot(ta, xgradcamplt[0,:], label='TDX1')
    # plt.plot(ta, amp+xgradcamplt[1,:], label='TDX2')
    # # plt.plot(ta, 2*amp+xgradcamplt[2,:], label='TDX3')
    # # plt.plot(ta, 3*amp+xgradcamplt[3,:], label='TDX4')
    # # plt.plot(ta, 4*amp+xgradcamplt[4,:], label='TDX5')
    # plt.legend(loc='upper right')
    # plt.plot(ta,salimap[0,:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, salimap[0,:],0,
    #         color='red',alpha=0.1)
    # plt.plot(ta,amp+salimap[1,:], color='red', label='Saliency-Map')
    # plt.fill_between(ta, amp+salimap[1,:],amp,
    #         color='red',alpha=0.1)
    # # plt.plot(ta,2*amp+salimap[2,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 2*amp+salimap[2,:],2*amp,
    # #         color='red',alpha=0.1)
    # # plt.plot(ta,3*amp+salimap[3,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 3*amp+salimap[3,:],3*amp,
    # #         color='red',alpha=0.1)
    # # plt.plot(ta,4*amp+salimap[4,:], color='red', label='Saliency-Map')
    # # plt.fill_between(ta, 4*amp+salimap[4,:],4*amp,
    # #         color='red',alpha=0.1)
    # plt.xlabel('Time Axis')
    # plt.ylabel('Pressure [-]')
    # plt.ylim(-amp,2*amp)
    # plt.yticks([-amp,0,amp,2*amp],[f"{-amp}","0","0",f"{amp}"])
    # # plt.ylim(-amp,5*amp)
    # # plt.yticks([-amp,0,amp,2*amp,3*amp,4*amp,5*amp],[f"{-amp}","0","0","0","0","0",f"{amp}"])
    # plt.xlim(0,W)
    # plt.grid(True)
    # # plt.xlim(35,40)
    # plt.tight_layout()
    # new_save_path = os.path.join(odir, f"saliency{k}.png")
    # plt.savefig(new_save_path)
    # plt.close()

res_list=np.array(res_list)
coefs_list=np.array(coefs_list)
reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"res_list:{res_list}")
print(f"coefs_list:{coefs_list}")
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"res:{np.mean(res_list)}")
print(f"coefs:{np.mean(coefs_list)}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")

(315, 2, 2500)
(315,)
torch.Size([315, 2, 2500])
109569


/tmp/ipykernel_2655947/2685038901.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))


(48,)
(48,)
coefficient void: 0.8824124055828574
re void: 0.05639940174478198
109569
(48,)
(48,)
coefficient void: 0.8268349212650058
re void: 0.04836358986678468
109569
(48,)
(48,)
coefficient void: 0.7722871357632052
re void: 0.08717179392790182
109569
(48,)
(48,)
coefficient void: 0.8592132125335791
re void: 0.05236340955419495
109569
(48,)
(48,)
coefficient void: 0.7673884432768617
re void: 0.06572777424459388
109569
(48,)
(48,)
coefficient void: 0.8442819833311496
re void: 0.06772545756913823
109569
(48,)
(48,)
coefficient void: 0.8571095467766693
re void: 0.07162785696070464
109569
(48,)
(48,)
coefficient void: 0.8705517515826754
re void: 0.04631580151889192
109569
(48,)
(48,)
coefficient void: 0.8474515395119966
re void: 0.05801332471112415
109569


KeyboardInterrupt: 

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import os

x_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/x_train_two_raw_std.npy")
t_train=np.load("/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset/t_train_two_raw_std.npy")
bdir="/mnt/sdb/yyamaguchi/simulationB4/bubble_glass_rand/dataset"
odir=os.path.join(bdir,"outputs_2raw_std_nn")
wdir=os.path.join(odir,"weights")
ldir=os.path.join(odir,"logs")
device="cuda:0"
B=64
W=2500
lr=0.001
N=100

class SimpleNN(nn.Module):
    def __init__(self, input_length):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_length,4096)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4096,2)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x.squeeze(1)

print(x_train.shape)
x_train=x_train.reshape(x_train.shape[0],-1)
# x_train=x_train[:,[1,3,4],:]
print(x_train.shape)
x=torch.from_numpy(x_train).float()
t=torch.from_numpy(t_train).float()
# x=x.unsqueeze(1)
print(x.shape)
amp=10
res_list=[]
coefs_list=[]
reg_list=[]
coefg_list=[]
dataset=TensorDataset(x,t)
Nttl=len(dataset)
trasize=int(0.7*Nttl)
valsize=int(0.15*Nttl)
testsize=Nttl-trasize-valsize
trainsetall,testset=random_split(dataset,[trasize+valsize,testsize])
testloader=DataLoader(testset,batch_size=1,shuffle=False)

for k in range(10):
    trainset,valset=random_split(trainsetall,[trasize,valsize])
    trainloader=DataLoader(trainset,batch_size=B,shuffle=True,drop_last=True)
    valloader=DataLoader(valset,batch_size=B,shuffle=False)
    model=SimpleNN(W*C).to(device)

    criterion=nn.MSELoss()
    optimizer=optim.Adam(params=model.parameters(),lr=lr)
    tlosshistory=[]
    vlosshistory=[]

    for epoch in range(N):
        model.train()
        rloss=0.0
        for bx, by in trainloader:
            bx=bx.to(device)
            by=by.to(device)
            optimizer.zero_grad()
            o=model(bx)
            loss=criterion(o,by)
            l1n=sum(p.abs().sum() for p in model.parameters())
            l2n=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l1*l1n+l2*l2n
            loss.backward()
            optimizer.step()
            rloss=rloss+loss.item()*bx.size(0)
        eloss=rloss/len(trainloader.dataset)
        tlosshistory.append(eloss)

        model.eval()
        vrloss=0.0
        with torch.no_grad():
            for vx,vy in valloader:
                vx=vx.to(device)
                vy=vy.to(device)
                vo=model(vx)
                vloss=criterion(vo,vy)
                vrloss+=vloss.item()*vx.size(0)
        veloss=vrloss/len(valloader.dataset)
        vlosshistory.append(veloss)

    os.makedirs(odir,exist_ok=True)
    os.makedirs(wdir,exist_ok=True)
    os.makedirs(ldir,exist_ok=True)
    torch.save(model.state_dict(), os.path.join(wdir,f'model{k}.pth'))
    plt.figure()
    plt.plot(range(1, N + 1), [np.log(l) for l in tlosshistory], label='Train Log(Loss)')
    plt.plot(range(1, N + 1), [np.log(l) for l in vlosshistory], label='Validation Log(Loss)')
    plt.title('Learning Curve (Log Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Log(Loss)')
    plt.rcParams["font.size"] = 16
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve_log{k}.png'))
    plt.close()

    plt.figure()
    plt.plot(range(1, N + 1), tlosshistory, label='Train Loss')
    plt.plot(range(1, N + 1), vlosshistory, label='Validation Loss')
    plt.title('Learning Curve (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.rcParams["font.size"] = 20
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'learning_curve{k}.png'))
    plt.close()

    model.load_state_dict(torch.load(os.path.join(wdir,f'model{k}.pth')))
    model=model.to(device)
    model.eval()
    pred=[]
    targ=[]
    with torch.no_grad():
        for tx,ty in testloader:
            tx=tx.to(device)
            ty=ty.to(device)
            o=model(tx)
            pred.append(o.cpu())
            targ.append(ty.cpu())
    pred=torch.cat(pred,dim=0)
    targ=torch.cat(targ,dim=0)

    pred=pred.numpy()
    targ=targ.numpy()

    print(pred.shape)

    corr=np.corrcoef(targ[:,0],pred[:,0])
    coef=corr[0,1]
    print(f'coefficient solid: {coef}')
    # re=np.mean(np.abs(targ[:,0]-pred[:,0])/(targ[:,0]+1e-7))
    re=np.sqrt(np.sum(np.power(targ[:,0]-pred[:,0],2))/targ.shape[0])
    print(f're solid: {re}')
    res_list.append(re)
    coefs_list.append(coef)
    corr=np.corrcoef(targ[:,1],pred[:,1])
    coef=corr[0,1]
    print(f'coefficient void: {coef}')
    # re=np.mean(np.abs(targ[:,1]-pred[:,1])/(targ[:,1]+1e-7))
    re=np.sqrt(np.sum(np.power(targ[:,1]-pred[:,1],2))/targ.shape[0])
    print(f're void: {re}')
    reg_list.append(re)
    coefg_list.append(coef)

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,0], pred[:,0], alpha=0.6, marker='o', label='Solid fraction')
    plt.plot([-1, 1], [-1, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(-0.0, 0.05)
    plt.ylim(-0.0, 0.05)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_solid{k}.png'))
    plt.close()

    plt.figure(figsize=(8, 8))
    plt.scatter(targ[:,1], pred[:,1], alpha=0.6, marker='o', label='Void fraction')
    plt.plot([0, 1], [0, 1], 'r--', label='Ideal (y=x)')
    plt.title('Predicted vs Actual Values')
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.xlim(0, 0.4)
    plt.ylim(0, 0.4)
    plt.rcParams["font.size"] = 20
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(ldir, f'val_pred_vs_actual_void{k}.png'))
    plt.close()

res_list=np.array(res_list)
coefs_list=np.array(coefs_list)
reg_list=np.array(reg_list)
coefg_list=np.array(coefg_list)
print(f"res_list:{res_list}")
print(f"coefs_list:{coefs_list}")
print(f"reg_list:{reg_list}")
print(f"coefg_list:{coefg_list}")
print(f"res:{np.mean(res_list)}")
print(f"coefs:{np.mean(coefs_list)}")
print(f"reg:{np.mean(reg_list)}")
print(f"coefg:{np.mean(coefg_list)}")


(315, 2, 2500)
(315, 5000)
torch.Size([315, 5000])


NameError: name 'l1' is not defined